#### Importing packages

In [1]:
import pandas as pd
from datetime import datetime, timedelta

import math
import glob

from matplotlib.colors import LinearSegmentedColormap
cmap = LinearSegmentedColormap.from_list(name='stock', colors=['red','white','green'])

#### Functions

In [2]:
def absolute_return(start, end):
    return round(100*(end-start)/start, 1)

def annualized_return(opening_value, closing_value, year):
    return round(100*(math.pow(closing_value/opening_value, 1/year) - 1), 1)

def helper(path):
    ## reading nifty50 stocks historical data
    l = [pd.read_csv(filename) for filename in glob.glob("{}/*.csv".format(path))]
    df = pd.concat(l, axis=0)

    ## processing date column
    df['date'] = df["date"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d").date())

    ## removing dates for which closing price is not populated
    df = df[~df['close'].isnull()]

    ## filter: last 10 years
    df = df[df['date'] >= datetime(2009, 12, 31).date()]

    ##* YF does not have last few days data of year 2009 therefore using '2010-01-04' as the closing data of year 2009
    df['date'] = np.where(df['date'] == datetime(2010, 1, 4).date(), datetime(2009, 12, 31).date(), df['date'])

    ## enriching year column
    df["year"] = df["date"].apply(lambda x: x.year)

    ## closing dates of each year
    closing_dates = list(df.groupby(['year'])['date'].agg(['max'])['max'].values)

    ## filter: closing dates
    df = df[df['date'].isin(closing_dates)]
    
    ## analysis
    analysis_df_1 = df.pivot_table(index='symbol', columns='year', values='close')

    ## yearly returns
    for year in range(2010, 2020):
        analysis_df_1[str(year)] = analysis_df_1.apply(lambda x: absolute_return(x[year-1], x[year]), axis=1)

    ## CAGR
    analysis_df_1["CAGR (1 Yr)"] = analysis_df_1["2019"]  
    analysis_df_1["CAGR (2 Yr)"] = analysis_df_1.apply(lambda x: annualized_return(x[2017], x[2019], 2), axis=1)
    analysis_df_1["CAGR (5 Yr)"] = analysis_df_1.apply(lambda x: annualized_return(x[2014], x[2019], 5), axis=1)
    analysis_df_1["CAGR (10 Yr)"] = analysis_df_1.apply(lambda x: annualized_return(x[2009], x[2019], 10), axis=1)

    ## overall returns (CAGR (1 yr + 2 yr + 5 yr + 10 yr)/4)
    analysis_df_1["overall_return"] = (analysis_df_1["CAGR (1 Yr)"] + analysis_df_1["CAGR (2 Yr)"] + analysis_df_1["CAGR (5 Yr)"] + analysis_df_1["CAGR (10 Yr)"])/4
    
    ## removing some columns
    analysis_df_1 = analysis_df_1[[col_ for col_ in analysis_df_1.columns if isinstance(col_, str)]]

    ## sorting
    analysis_df_1.sort_values(['overall_return', 'CAGR (5 Yr)', 'CAGR (2 Yr)', 'CAGR (1 Yr)'], ascending=[False, False, False, False], inplace=True)
    
    return analysis_df_1

$Overall Return\ (\%) = \frac{CAGR\ (1 Yr.)\ +\ CAGR\ (2 Yr.)\ +\ CAGR\ (5 Yr.)\ +\ CAGR\ (10 Yr.)}{4}$

#### Historical Returns - NIFTY50

In [8]:
nifty50_df = helper("data/yahoo_finance_nifty50_stocks_data/")

## adding style to dataframe
nifty50_df = nifty50_df.style.background_gradient(cmap=cmap, vmin=-200, vmax=200).render()

In [12]:
display(HTML(nifty50_df))

year,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,CAGR (1 Yr),CAGR (2 Yr),CAGR (5 Yr),CAGR (10 Yr),overall_return
symbol,,,,,,,,,,,,,,,
BAJFINANCE,105.100000,-13.900000,123.100000,20.400000,120.300000,72.500000,40.100000,108.600000,50.600000,60.100000,60.100000,55.300000,64.800000,62.300000,60.625000
BAJAJFINSV,28.800000,-7.300000,114.300000,-17.800000,76.100000,52.000000,45.600000,80.700000,23.800000,45.000000,45.000000,34.000000,48.300000,38.800000,41.525000
KOTAKBANK,10.300000,-5.100000,51.000000,12.000000,73.600000,13.900000,-0.000000,40.400000,24.400000,34.100000,34.100000,29.100000,21.700000,23.400000,27.075000
NESTLEIND,52.900000,7.400000,21.900000,5.900000,20.700000,-8.700000,3.400000,30.600000,40.800000,33.400000,33.400000,37.100000,18.300000,19.500000,27.075000
ICICIBANK,30.200000,-40.200000,66.300000,-3.500000,60.700000,-26.000000,-2.300000,35.300000,14.700000,49.600000,49.600000,31.000000,10.900000,12.900000,26.100000
TITAN,152.900000,-4.900000,66.300000,-19.300000,66.300000,-9.000000,-5.900000,162.600000,8.500000,27.600000,27.600000,17.700000,25.500000,32.500000,25.825000
RELIANCE,-1.600000,-34.500000,21.200000,6.600000,-0.500000,13.900000,6.700000,70.200000,21.700000,35.000000,35.000000,28.200000,27.700000,10.900000,25.450000
ASIANPAINT,60.900000,-9.900000,71.000000,10.500000,53.500000,17.400000,0.800000,30.000000,18.500000,30.000000,30.000000,24.100000,18.900000,25.900000,24.725000
HDFC,36.200000,-10.500000,27.100000,-1.300000,0.000000,54.500000,-0.100000,35.500000,15.100000,22.600000,22.600000,18.800000,24.100000,16.300000,20.450000


#### Historical Returns - NIFTY Midcap 50

In [8]:
nifty_midcap50_df = helper("data/yahoo_finance_nifty_midcap50_stocks_data/")

## adding style to dataframe
nifty_midcap50_df = nifty_midcap50_df.style.background_gradient(cmap=cmap, vmin=-200, vmax=200)

In [12]:
nifty_midcap50_df

year,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,CAGR (1 Yr),CAGR (2 Yr),CAGR (5 Yr),CAGR (10 Yr),overall_return
symbol,,,,,,,,,,,,,,,
SRF,66.700000,-32.400000,-16.000000,10.400000,294.800000,44.500000,22.300000,28.100000,0.900000,71.800000,71.800000,31.700000,31.500000,32.100000,41.775000
BATAINDIA,84.400000,46.300000,63.600000,21.500000,24.000000,-20.100000,-13.700000,65.900000,51.800000,54.400000,54.400000,53.100000,21.800000,33.400000,40.675000
CHOLAFIN,179.300000,-40.300000,138.100000,-9.300000,93.900000,35.500000,47.500000,37.300000,-3.100000,21.300000,21.300000,8.400000,26.400000,36.500000,23.150000
BALKRISIND,16.300000,20.500000,91.600000,13.500000,80.000000,7.400000,70.300000,116.600000,-22.600000,7.200000,7.200000,-8.900000,26.900000,33.500000,14.675000
VOLTAS,24.100000,-66.300000,44.200000,9.600000,109.000000,33.700000,1.200000,99.800000,-15.600000,19.100000,19.100000,0.300000,22.100000,14.200000,13.925000
RAMCOCEM,-7.900000,-3.300000,139.400000,-22.100000,80.100000,12.800000,40.800000,43.400000,-18.500000,17.900000,17.900000,-2.000000,16.900000,20.700000,13.375000
ADANIPOWER,31.700000,-52.100000,-1.200000,-36.900000,13.700000,-27.200000,-6.800000,38.800000,22.200000,21.300000,21.300000,21.700000,6.900000,-4.600000,11.325000
APOLLOHOSP,31.100000,24.200000,39.300000,20.200000,19.400000,29.800000,-19.500000,2.100000,4.400000,14.600000,14.600000,9.400000,5.000000,15.300000,11.075000
MRF,17.200000,-3.100000,83.700000,51.100000,95.800000,5.200000,22.600000,48.000000,-7.400000,-1.000000,-1.000000,-4.300000,11.800000,26.900000,8.350000


#### Historical Returns - NIFTY Smallcap 50

In [10]:
nifty_smallcap50_df = helper("data/yahoo_finance_nifty_smallcap50_stocks_data/")

## adding style to dataframe
nifty_smallcap50_df = nifty_smallcap50_df.style.background_gradient(cmap=cmap, vmin=-200, vmax=200)

In [13]:
nifty_smallcap50_df

year,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,CAGR (1 Yr),CAGR (2 Yr),CAGR (5 Yr),CAGR (10 Yr),overall_return
symbol,,,,,,,,,,,,,,,
NAVINFLUOR,-14.100000,-3.400000,3.700000,-10.400000,133.500000,205.800000,34.900000,64.300000,-15.800000,44.100000,44.100000,10.100000,52.400000,30.900000,34.375000
CANFINHOME,33.500000,-25.900000,72.900000,14.000000,187.500000,134.900000,46.600000,49.500000,-38.900000,36.100000,36.100000,-8.800000,33.800000,37.400000,24.625000
GODFRYPHLP,6.400000,21.400000,38.600000,-25.400000,14.800000,128.800000,-25.300000,-3.400000,-10.100000,43.400000,43.400000,13.500000,16.300000,12.600000,21.450000
PVR,-21.300000,-11.900000,116.300000,131.400000,8.000000,14.400000,43.300000,22.800000,13.700000,18.300000,18.300000,16.000000,22.100000,26.100000,20.625000
GRANULES,4.300000,-35.600000,158.000000,23.500000,325.300000,80.000000,-27.400000,27.200000,-34.700000,37.200000,37.200000,-5.400000,8.300000,29.800000,17.475000
VIPIND,280.900000,-42.300000,9.200000,-23.600000,76.700000,-7.900000,11.900000,199.700000,47.000000,-16.400000,-16.400000,10.800000,30.600000,28.500000,13.375000
SUNTECK,-0.100000,-44.900000,55.300000,-35.600000,-7.500000,-16.300000,-7.400000,286.000000,-17.300000,20.300000,20.300000,-0.200000,24.400000,4.200000,12.175000
DCMSHRIRAM,-19.400000,-21.600000,71.100000,-18.800000,185.100000,-12.100000,53.500000,160.600000,-39.300000,16.700000,16.700000,-15.800000,20.000000,20.100000,10.250000
KAJARIACER,36.900000,35.600000,131.400000,34.700000,89.800000,65.700000,-5.200000,56.600000,-32.400000,6.500000,6.500000,-15.100000,12.100000,34.600000,9.525000


#### References

- https://www.moneycontrol.com/news/business/markets/these-12-nifty-stocks-gave-double-digit-cagr-in-10-years-are-they-good-for-your-portfolio-4847181.html
- https://getmoneyrich.com/historical-returns-given-top-indian-stocks/
- https://www.marketwatch.com/story/these-are-the-20-best-performing-stocks-of-the-past-decade-and-some-of-them-will-surprise-you-2019-12-09

In [ ]:
#no